In [1]:
# %matplotlib notebook
# import geopandas as gpd
# import pandas as pd
# import matplotlib.pyplot as plt
# from tqdm import tqdm
# import contextily as cx
import xml.etree.ElementTree as ET
# from shapely import wkt
import random
import sumolib.net as sumonet
import pandas as pd
from tqdm import tqdm

# Changing weights in TAZs

In [9]:
net = sumonet.readNet('Seattle_road_network.net.xml')
edges = net.getEdges()
edge_p_dict = {}
for edge in edges:
    ID = edge.getID()
    p = edge.getPriority()
    edge_p_dict[ID] = p

In [11]:
edges_in_TAZs = []

# load sumo tazs
tree = ET.parse('new_Taz_with_pseudo_link.add.xml')
root = tree.getroot()
sumo_taz = []
for taz in root.findall('taz'):
#     if str(taz.get('id'))>5000:
    for src in taz.findall('tazSource'):
        link_id = src.get('id')
        edges_in_TAZs.append(link_id)
        link_priority = edge_p_dict[link_id]
        if src.get('weight')!= '0.00':
            src.set('weight', str(link_priority)+'.00')
    for sink in taz.findall('tazSink'):
        link_id = sink.get('id')
        edges_in_TAZs.append(link_id)
        link_priority = edge_p_dict[link_id]
        if sink.get('weight')!= '0.00':
            sink.set('weight', str(link_priority)+'.00')



In [ ]:
# Save the modified XML back to a file
tree.write('modified_new_Taz_with_pseudo.add.xml', encoding='utf-8', xml_declaration=True)

In [14]:
len(edges)

7384

In [15]:
len(set(edges_in_TAZs))

4031

In [16]:
with open('edges_in_TAZs.txt', 'w') as f:
    for edge in set(edges_in_TAZs):
        f.write(edge+'\n')

# reducing speed of alleys

In [14]:
# load sumo tazs
tree = ET.parse('Seattle_road_network.net.xml')
root = tree.getroot()
# sumo_taz = []
for edge in root.findall('edge'):
    if edge.get('function') in['walkingarea', 'internal', 'crossing']:
        continue
    else:
        if type(edge.get('priority'))==str:
            if int(edge.get('priority'))==2:
                for lane in edge.findall('lane'):
                    if float(lane.get('speed'))==6.71:
                        lane.set('speed', str(4.47))  # 15mph->10mph
                    if float(lane.get('speed'))==5.56:
                        lane.set('speed', str(3.35))  # 12.4mph->7.5mph
        else:
            print(edge.get('id'))


In [15]:
# Save the modified XML back to a file
tree.write('new_Seattle_road_network.net.xml', encoding='utf-8', xml_declaration=True)

# deleting some pedestrians after deleting alleys from network

In [6]:
net = sumonet.readNet('no_alley_Seattle_road_network.net.xml')
edges = net.getEdges()

In [7]:
# load sumo tazs
tree = ET.parse('ped_od_input.rou.xml')
root = tree.getroot()
for person in (root.findall('person')):
    for walk in person.findall('walk'):
        if (walk.get('from') in edges) and (walk.get('to') in edges):
            continue
        else:
            root.remove(person)
            

In [8]:
# Save the modified XML back to a file
tree.write('new_ped_od_input.rou.xml', encoding='utf-8', xml_declaration=True)

# filtering demnad

In [20]:
# total number of trips?
tree = ET.parse('new_psrc_vehicle_trips_converted_taz.rou.xml')
root = tree.getroot()
print(len(root.findall('trip')))

166684


In [18]:
# out of downtown to out of downton trips
tree = ET.parse('new_psrc_vehicle_trips_converted_taz.rou.xml')
root = tree.getroot()
counter = 0
for trip in (root.findall('trip')):
    if (int(trip.get('fromTaz')) >= 5000) and (int(trip.get('toTaz')) >= 5000):
        counter += 1
        continue
    else:
        root.remove(trip)
print(counter)
# Save the modified XML back to a file
tree.write('new_psrc_vehicle_trips_converted_taz_no downtown.rou.xml', encoding='utf-8', xml_declaration=True)          

16950


In [17]:
#  downtown to downton trips
tree = ET.parse('new_psrc_vehicle_trips_converted_taz.rou.xml')
root = tree.getroot()
counter = 0
for trip in (root.findall('trip')):
    if (int(trip.get('fromTaz')) < 5000) and (int(trip.get('toTaz')) < 5000):
        counter+=1
        continue
    else:
        root.remove(trip)
print(counter)
# Save the modified XML back to a file
tree.write('new_psrc_vehicle_trips_converted_taz_just downtown.rou.xml', encoding='utf-8', xml_declaration=True)          

20357


In [33]:
# downtown to/from out of downtown trips
tree = ET.parse('new_psrc_vehicle_trips_converted_taz.rou.xml')
root = tree.getroot()
counter = 0
for trip in (root.findall('trip')):
    if (
        ((int(trip.get('fromTaz')) >= 5000) and (int(trip.get('toTaz')) < 5000)) 
        or ((int(trip.get('fromTaz')) < 5000) and (int(trip.get('toTaz')) >= 5000))
           ):
        counter+=1
        continue
    else:
        root.remove(trip)
print(counter)
# Save the modified XML back to a file
tree.write('new_psrc_vehicle_trips_converted_taz_one end downtown.rou.xml', encoding='utf-8', xml_declaration=True)          

129377


# Reducing demand

In [2]:
# tree = ET.parse('modified_new_psrc_vehicle_trips_converted_taz.rou.xml')
tree = ET.parse('od2trips_new_psrc_vehicle_trips_converted_taz.rou.xml') 
# make sure in this file, rows you want to change have the desired attribute (like 'fromTaz')
root = tree.getroot()

# reduce_taz = [5010, 5008, 5000, 5002]
# reduce_taz = {5010:30, 5002:20, 5000:5, 5008:15}
# reduce_taz = {5010:15, 5002:10}
# reduce_taz = {5002:5}
reduce_taz = {}
trip_dict = {key:[] for key, val in reduce_taz.items()}
print(f"initial number of trips: {len(root.findall('trip'))}")

initial number of trips: 166597


In [3]:
for trip in (root.findall('trip')):
    otaz = int(trip.get('fromTaz'))
    if otaz in reduce_taz.keys():
        trip_dict[otaz].append(trip)

In [4]:
for taz in trip_dict.keys():
    reduction_percent = reduce_taz[taz]
    print(f'taz: {taz}, reduction percentage: {reduction_percent}')
    reduction_number = int(reduction_percent/100 * len(trip_dict[taz]))
    for i in tqdm(range(reduction_number)):
        idx = random.randint(0, len(trip_dict[taz])-1)
        trip_to_remove = trip_dict[taz][idx]
        root.remove(trip_to_remove)
        trip_dict[taz].remove(trip_to_remove)


### moving from 5010 to 5000
for trips going to sumo_N TAZs: 

remove 10 percent and move another 30 percent to I5N

In [5]:
sr99N_sumoN = []
tobe_removed = 30
tobe_moved = 15

with open('sumo_south.txt', 'r') as f:
        content = f.read()
        sumoS_tazs = eval(content)



In [6]:
counter5000=0
counter5010=0
for trip in (root.findall('trip')):
    otaz = (trip.get('fromTaz'))
    if otaz is not None:
        otaz = int(otaz)
        if otaz==5010:
            dtaz = (trip.get('toTaz'))
            if (dtaz is not None) and (dtaz not in sumoS_tazs):
                dtaz = int(dtaz)
                sr99N_sumoN.append(trip)
                counter5010+=1
        if otaz==5000:
            counter5000+=1
print(counter5000)
print(counter5010)

26248
15909


In [7]:
reduction_number = int(tobe_removed/100 * len(sr99N_sumoN))
move_number = int(tobe_moved/100 * len(sr99N_sumoN))

for i in tqdm(range(reduction_number)):
    idx = random.randint(0, len(sr99N_sumoN)-1)
    trip_to_remove = sr99N_sumoN[idx]
    root.remove(trip_to_remove)
    sr99N_sumoN.remove(trip_to_remove)

for i in tqdm(range(move_number)):
    idx = random.randint(0, len(sr99N_sumoN)-1)
    trip_to_move = sr99N_sumoN[idx]
    trip_to_move.set('fromTaz', str(5000))
    sr99N_sumoN.remove(trip_to_move)

100%|████████████████████████████████████| 2386/2386 [00:00<00:00, 23319.44it/s]


In [8]:
counter5000=0
counter5010=0
for trip in (root.findall('trip')):
    otaz = int(trip.get('fromTaz'))
    if otaz==5010:
        dtaz = int(trip.get('toTaz'))
        if dtaz not in sumoS_tazs:
            counter5010+=1
    if otaz==5000:
        counter5000+=1
print(counter5000)
print(counter5010)

28634
7144


In [9]:
print(f"final number of trips: {len(root.findall('trip'))}")
# tree.write('reduced3_new_psrc_vehicle_trips_converted_taz.rou.xml', encoding='utf-8', xml_declaration=True)
tree.write('reduced_od2trips_new_psrc_vehicle_trips_converted_taz.rou.xml', encoding='utf-8', xml_declaration=True)

final number of trips: 161825


# remove selected starting edges from od2trips results
First, make sure you added vehicle type to start of that file

In [10]:
tree = ET.parse('reduced_od2trips_new_psrc_vehicle_trips_converted_taz.rou.xml')
root = tree.getroot()

north_TAZs = [442, 440, 438, 436, 434, 432, 430]  # see limitations in the doc

from_tazs= [636, 5000, 5004, 5010, 5008, 6001, 7001, 7002, 7003, 7004, 7005] + north_TAZs  # all??
to_tazs = [636, 5005, 5009, 5011, 6001, 7001, 7002, 7004, 7005] + north_TAZs

for trip in (root.findall('trip')):
    otaz = int(trip.get('fromTaz'))
    dtaz = int(trip.get('toTaz'))
    if otaz in from_tazs and dtaz in to_tazs:
        del trip.attrib['to']
        del trip.attrib['from']
        
    elif otaz in from_tazs:
        del trip.attrib['from']
        del trip.attrib['toTaz']
        
    elif dtaz in to_tazs:
        del trip.attrib['to']
        del trip.attrib['fromTaz']
        
    else:
        del trip.attrib['toTaz']
        del trip.attrib['fromTaz']
        

In [11]:
tree.write('adjusted_reduced_od2trips_new_psrc_vehicle_trips_converted_taz.rou.xml', encoding='utf-8', xml_declaration=True)

# Speed and departLane and ArrivalLane

11.18 - 25
13.41 - 30
15.65 - 35
17.88 - 40